In [17]:
#We are taking prediction results of a district, taken at different point of time (different months within a year). We are going to combine all prediction results to make a final prediction for that year.

# import all required packages
from PIL import Image
import math
from PIL import Image
from scipy import misc
from scipy import ndimage
import pandas as pd
import unittest
import os, sys
import numpy as np

In [30]:
'''
Change districts, and years accordingly
''' 
districts = ['Bangalore','Chennai','Delhi','Gurgaon','Hyderabad','Kolkata','Mumbai']
years=['2016','2017','2018','2019']

In [24]:
'''
Covert monthly predictions from .tif to .png
'''
for district in districts:
    main_folder = 'Classification_'+district     #we have stored monthly predections in a folder named by district name
    print(district)
    os.makedirs(main_folder+"/pngs",exist_ok=True)
    for infile in os.listdir(main_folder):
        if infile[-4:] == ".tif":                   #reading all tif files in given folder
            im = Image.open(main_folder+"/"+infile)
            im.save(main_folder+"/pngs/"+infile[:-4]+'.png')

Kolkata


In [25]:
# Defining all required functions
'''
Rule-based post-classification correction

merge the monthly predictions and overall median prediction to get a single prediction for year
Pixel value 0 denotes Background, 1 denotes greenery, 2 denotes water, 3 benotes Built-Up, 4 denotes Barren land
Input-
1) x = list of predicted value of a particular pixel in the best 5 months of a particular year
2) median_pixel_prediction = predicted value of the above pixel over the year median image
'''  
def merge_prediction(yearly_pixel_predictions, median_pixel_prediction):    
    total_predictions = len(yearly_pixel_predictions)
    
    #find the count of each kind of pixel value for a pixel across all the given years
    background_count = yearly_pixel_predictions.count(0)
    green_count = yearly_pixel_predictions.count(1)
    water_count = yearly_pixel_predictions.count(2)
    builtup_count = yearly_pixel_predictions.count(3)
    barrenland_count = yearly_pixel_predictions.count(4)
    
    #Applying different rules for post-classification error correction
    
    # Rule1: If pixel is predicted as background in all 5 months, consider it background for the entire year
    if (background_count == total_predictions):
        return '0'
    
    # Rule2: If pixel is predicted as water more times than green in 5 months, consider it water for the entire year
    elif (water_count > 0 and green_count > 0 and water_count > 1.5 * green_count):
        return '2'
    
    # Rule3: If pixel is predicted as water more than 50% times, consider it water for the entire year
    elif (water_count >= 0.5 * total_predictions):
        return '2'
    
    # Rule4: If pixel is predicted as green more times than water in 5 months, consider it green for the entire year
    elif (water_count>0 and green_count>0 and water_count <= 1.5 * green_count ):
        return '1'
    
    # Rule5: This rule helps to eliminate shadows which are mis-interpreted as water
    elif (water_count != 0 and green_count == 0 ):
        return str(median_pixel_prediction)
    
    # Rule6: After identifying water, if a pixel is predicted as green atleast once, consider it green for the entire year
    elif ( green_count >=1 ):
        return '1'
    
    # Rule7: If a pixel is neither green nor water, then it is barrenland or builtup as per majority for the entire year
    elif((barrenland_count > builtup_count) and green_count==0 and water_count==0 ):
        return '4'
    
    # Rule8: If a pixel is neither green nor water, then it is barrenland or builtup as per majority for the entire year
    else:
        return '3'
    
    

In [26]:
'''
Read all the different predictions of a district. 
Out of the 12 months for each year, we see which months have the maximum number of green pixels (Assuming them to be most accurate). 
The top 5 months are chosen to create the final pixel-level predictions for each year.
'''
for district in districts:
    print(district)
    main_folder = 'Classification_'+district     #we have downloaded monthly predections in a folder named by district name from google drive
    os.makedirs(main_folder+"/final",exist_ok=True)
    for year in years:
        print(year)
        Green_pixs = []  #This list will store the key-value pairs where key=month and value=number of green pixels predicted in that month
        for infile in os.listdir(main_folder+"/pngs/"):
            if 'Classification_'+district in infile and year in infile and 'median' not in infile: #reading all monthly predictions
                im = Image.open(main_folder+"/pngs/"+infile)
                month = infile[-6:-4]
                Green_pixs.append((month,np.unique(im,return_counts=True)[1][1]))
                
        Green_pixs.sort(key=lambda x:x[1],reverse=True) #sort the dictionary by values 
        
        best_months=[Green_pixs[i][0] for i in range(min(5,len(Green_pixs)))] #taking best 5 months on the basis of greenery
        print("The best 5 months this year are: ",best_months)
            
        best_month1_path = main_folder+'/pngs/'+'Classification_'+district+'_'+year+'_30mtr_'+best_months[0]+'.png'
        best_month2_path = main_folder+'/pngs/'+'Classification_'+district+'_'+year+'_30mtr_'+best_months[1]+'.png'
        best_month3_path = main_folder+'/pngs/'+'Classification_'+district+'_'+year+'_30mtr_'+best_months[2]+'.png'
        best_month4_path = main_folder+'/pngs/'+'Classification_'+district+'_'+year+'_30mtr_'+best_months[3]+'.png'
        best_month5_path = main_folder+'/pngs/'+'Classification_'+district+'_'+year+'_30mtr_'+best_months[4]+'.png'
        year_median_path = main_folder+'/pngs/'+'Classification_'+district+'_'+year+'_30mtr_year_median.png'
        
        # Read all the required predictions in an array
        best_prediction1 = np.asarray(Image.open(best_month1_path)) 
        best_prediction2 = np.asarray(Image.open(best_month2_path))
        best_prediction3 = np.asarray(Image.open(best_month3_path))
        best_prediction4 = np.asarray(Image.open(best_month4_path))
        best_prediction5 = np.asarray(Image.open(best_month5_path))
        year_median_prediction = np.asarray(Image.open(year_median_path))

        print("best prediction1: ",np.unique(best_prediction1,return_counts=True))
        print("best prediction2: ",np.unique(best_prediction2,return_counts=True))
        print("best prediction3: ",np.unique(best_prediction3,return_counts=True))
        print("best prediction4: ",np.unique(best_prediction4,return_counts=True))
        print("best prediction5: ",np.unique(best_prediction5,return_counts=True))
        print("median prediction: ",np.unique(year_median_prediction,return_counts=True))
                
        image_dimension = best_prediction1.shape
        #print(image_dimension)

        #Initializing the final prediction matrix for a particular year
        final_prediction = np.zeros(image_dimension[0] * image_dimension[1]).reshape(image_dimension)
        #print(final_prediction)

        for i in range(image_dimension[0]):
            for j in range(image_dimension[1]):
                x = [ best_prediction1[i][j], best_prediction2[i][j], best_prediction3[i][j], best_prediction4[i][j], best_prediction5[i][j] ]
                final_prediction[i,j] = merge_prediction(x, year_median_prediction[i][j])

        print("final_prediction ",np.unique(final_prediction,return_counts=True))
        
        final_prediction = (Image.fromarray(final_prediction)).convert("L")
        final_prediction.save(main_folder+'/final/'+district+'_prediction_'+year+'.png')
        
print("Done!")



Kolkata
2017
[('10', 39509)]
[('10', 39509), ('02', 10286)]
[('10', 39509), ('02', 10286), ('01', 1951)]
[('10', 39509), ('02', 10286), ('01', 1951), ('11', 44193)]
[('10', 39509), ('02', 10286), ('01', 1951), ('11', 44193), ('03', 21415)]
[('10', 39509), ('02', 10286), ('01', 1951), ('11', 44193), ('03', 21415), ('04', 27269)]
[('10', 39509), ('02', 10286), ('01', 1951), ('11', 44193), ('03', 21415), ('04', 27269), ('09', 17201)]
[('10', 39509), ('02', 10286), ('01', 1951), ('11', 44193), ('03', 21415), ('04', 27269), ('09', 17201), ('12', 4865)]
[('10', 39509), ('02', 10286), ('01', 1951), ('11', 44193), ('03', 21415), ('04', 27269), ('09', 17201), ('12', 4865), ('06', 32536)]
The best 5 months this year are:  ['11', '10', '06', '04', '03']
best prediction1:  (array([0, 1, 2, 3, 4], dtype=int32), array([143324,  44193,  51120,   8749,   7630]))
best prediction2:  (array([0, 1, 2, 3, 4], dtype=int32), array([143324,  39509,  28441,  39522,   4220]))
best prediction3:  (array([0, 1, 2,

In [31]:
'''
Color coding the final prediction maps for a particular year. Uptill here the pixel values contain greyscale values. For easier visualization, we assign intuitive colors to different land-cover classes. These images will be stored at final/Color_coded_final_predictions subfolder for each district 
'''
for district in districts:
    print(district)
    main_folder = 'Classification_'+district+"/final"     #we have downloaded monthly predections in a folder named by district name from google drive
    os.makedirs(main_folder+"/Color_coded_final_predictions",exist_ok=True)
    for year in years:
        image_path = main_folder+'/'+district+'_prediction_'+year+'.png'  
        img = Image.open(image_path)
        img = img.convert("RGBA")
        pixdata = img.load()
        #print(img.getcolors()) #use this command to visualize already assigned colors to each label
        
        for y in range(img.size[1]):
            for x in range(img.size[0]):
                if pixdata[x, y] == (0, 0, 0, 255):      # background 
                    pixdata[x, y] = (0,0,0,0)            # black color
                elif pixdata[x, y] == (1, 1, 1, 255):    # green
                    pixdata[x, y] = (34,139,34, 255)     # green color
                elif pixdata[x, y] == (2, 2, 2, 255):    # water
                    pixdata[x, y] = (2, 4, 251, 255)     # blue color
                elif pixdata[x, y] == (3, 3, 3, 255):    # built-up 
                    pixdata[x, y] = (255, 255, 102, 255) # yellow color
                elif pixdata[x, y] == (4, 4, 4, 255):    # bareland
                    pixdata[x, y] = (255, 80, 80, 255)   # red color

        img.save(main_folder+"/Color_coded_final_predictions/"+district+'_colored_prediction_'+year+'.png')
        print("Done")




Kolkata
Done
Done
Done
Done
